# Transfer Attack on CIFAR10

In [1]:
import sys

import torch
import torch.nn as nn

sys.path.insert(0, '..')
import torchattacks

## Load source model and data

In [2]:
sys.path.insert(0, '..')
import robustbench
from robustbench.data import load_cifar10
from robustbench.utils import load_model, clean_accuracy

images, labels = load_cifar10(n_examples=5)
print('[Data loaded]')

device = "cuda"
model = load_model('Wong2020Fast', norm='Linf').to(device)
acc = clean_accuracy(model, images.to(device), labels.to(device))
print('[Model loaded]')
print('Acc: %2.2f %%'%(acc*100))

Files already downloaded and verified
[Data loaded]
[Model loaded]
Acc: 100.00 %


In [3]:
from torchattacks import PGD
atk = PGD(model, eps=8/255, alpha=2/225, steps=10, random_start=True)
atk.save(data_loader=[(images, labels)], save_path="_transfer.pt")

- Save progress: 100.00 % / Robust accuracy: 60.00 % / L2: 1.64071 (0.180 it/s) 	


## Load target model and adv data

In [4]:
adv_loader = atk.load(load_path="_transfer.pt")
adv_images, labels = iter(adv_loader).next()

Data is loaded in the following order: [adv_inputs, labels]


In [5]:
model = load_model('Standard', norm='Linf').to(device)
print('[Model loaded]')
acc = clean_accuracy(model, adv_images.to(device), labels.to(device))
print('Acc: %2.2f %%'%(acc*100))

[Model loaded]
Acc: 60.00 %
